In [1]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils import configuration

settings = configuration.settings
api_key = settings.api_key

In [2]:
def get_data_filename(filename='json_data'):
    return os.path.join('assets', filename+".json")


def write_json(data, filename='json_data'):
    json_asset = get_data_filename(filename)
    # Directly from dictionary
    json_string = json.dumps(data)
    with open(json_asset, 'w') as outfile:
        json.dump(json_string, outfile)

    # Using a JSON string
    with open(json_asset, 'w') as outfile:
        outfile.write(json_string)
        outfile.close()


def read_json(filename='json_data'):
    json_asset = get_data_filename(filename)
    try:
        with open(json_asset) as json_file:
            data = json.load(json_file)
            json_file.close()
            return data
    except FileNotFoundError:
        print(f"{filename} not found.")

In [3]:
watcher = TftWatcher(api_key)

my_region = 'na1'
my_summoner = 'fury hawkx'

In [4]:
me = watcher.summoner.by_name(my_region, my_summoner)
my_ranked_stats = watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

[{'leagueId': '9396da90-a642-44bb-b84c-d5fae367b151', 'queueType': 'RANKED_TFT', 'tier': 'GOLD', 'rank': 'III', 'summonerId': 'jg7S93hSw4KfuYyaks9xYzAFZ4w3bp_PSfT4LsHJi60A-SQ', 'summonerName': 'fury hawkx', 'leaguePoints': 21, 'wins': 8, 'losses': 2, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}]


In [7]:
me

{'id': 'jg7S93hSw4KfuYyaks9xYzAFZ4w3bp_PSfT4LsHJi60A-SQ',
 'accountId': 'Ab7L0SV8v_nX7hVWwKV7IbEAbA0dj-C4e9PvMcPaU-7hf8o',
 'puuid': 'K8izf-1-ovMV0xlJ2qhoL7Amw98UG248C2xf8GGDVSqf73KklNrVoNN1PQe1R4Zb2JFsOFa_kSEzpg',
 'name': 'fury hawkx',
 'profileIconId': 745,
 'revisionDate': 1657092043618,
 'summonerLevel': 20}

In [5]:
my_matches = watcher.match.by_puuid(my_region, me['puuid'], count=100)
write_json(my_matches, filename='my_matches'+'_'+my_summoner)

In [6]:
last_match = watcher.match.by_id(my_region, my_matches[0])
write_json(last_match, filename='last_match')

In [7]:
tft_assets_url = 'https://raw.communitydragon.org/latest/cdragon/tft/en_us.json'

r = requests.get(tft_assets_url)
tft_assets = r.json()
write_json(tft_assets, filename='tft_assets')

In [8]:
matches_assets = json.loads('[]')

for idx, match in enumerate(my_matches[:]):
    match_detail = watcher.match.by_id(my_region, match)
    matches_assets.append(match_detail)

    time.sleep(5)

write_json(matches_assets, filename='matches_detail'+'_'+my_summoner)

In [98]:
challengers = watcher.league.challenger(my_region)
write_json(challengers, filename='challengers')

In [111]:
sorted(challengers['entries'], key=lambda item: item['leaguePoints'], reverse=True)

[{'summonerId': '_U4GJSGv-DEG7MKfJKLmsV-mvefxXMzApJ4tGa_nRQv5sUk',
  'summonerName': 'sètsuko',
  'leaguePoints': 1741,
  'rank': 'I',
  'wins': 199,
  'losses': 95,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'summonerId': 'F076SSJmWsKsPULh8XD2yr_CcrGzmberIL9oH6WiQ4sydzT6',
  'summonerName': 'Dıshsoap',
  'leaguePoints': 1483,
  'rank': 'I',
  'wins': 162,
  'losses': 58,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': True},
 {'summonerId': 'mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0',
  'summonerName': 'BC Socks',
  'leaguePoints': 1347,
  'rank': 'I',
  'wins': 143,
  'losses': 65,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': True},
 {'summonerId': 'Gzedh6XF2T6y35wuGW5eKa0MQE8iyGpNJwj2mSMZAepcl2f-',
  'summonerName': 'C9 k3soju',
  'leaguePoints': 1235,
  'rank': 'I',
  'wins': 218,
  'losses': 145,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStre

In [117]:
summoners_challengers = json.loads('[]')

for idx, challenger in enumerate(challengers['entries'][:]):
    summoner = watcher.summoner.by_id(my_region, challenger['summonerId'])
    summoners_challengers.append(summoner)

    time.sleep(1)

write_json(summoners_challengers, filename='summoners_challengers')


In [122]:
summoners_challengers_df = pd.json_normalize(summoners_challengers)
challengers_df = pd.json_normalize(challengers['entries'])

In [129]:
summoners_challengers_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,BcLHPFVnuMFT_g6cJdB7LsDocnWKJ3TQyp9mMIY7amMcq-...,St_wL0UOxf_11zObuZn-7FE-yGRGjfjFaeHlAbCMCESgyq...,Spethom,4904,1656316118401,197
1,rCmZ-jOb9GFbAkeHOF72Dx8H5lgTi-hyvCx2OIx9vNnMCKg,72e-TFAU4NDDJv0ewfeiKQGwD6Xp1Ty9RvNhzzvrqxz6bA,X_u88wM67NMBXYLCbCAGNCx_19T7pat-j71dhydlbPYF19...,dallasyes,7,1656311568160,53
2,5PP1PByW13OTIHNs8f_MQbI2utzf-CKbFa3imrCUD-5JM-k,CE4kJ6tt7PwB4X24Y8CXS4pV73kqdSvJ0wWYyFi_ypioqA,SYeHG18d6HblklTVFAcDoGsAXWwTq1ZeflhRbaztiP900s...,prestivent,4026,1656296968000,128
3,F076SSJmWsKsPULh8XD2yr_CcrGzmberIL9oH6WiQ4sydzT6,5CXmlT14PxMEPUOluZZ6VnG0iyHHkcLFtbhFSdpDzWUNWQ...,miECR0OOaJlxGXdFIC3_icU8aPZ9SCTmzrQuuMkLvo64Zx...,Dıshsoap,4368,1656302298000,17
4,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,vNJAtsWbXGLyDPv5HWK6T2VsVvgaY3O7LOUNvL23vJt1-Q,hkhRHja9YNEGxs-c2YZxHJmvTcZaPQSaAMHWnbYt0qwlEC...,Pawnup,659,1656311139000,140
...,...,...,...,...,...,...,...
71,hpVP56ElzFpI9hJtd7l53WcBMs8s2kvqDEiWJnGCJVyLnSg,4Yvk3S970wp_QRtVv1yJ-TgydOOk_qZHnO_LscPrurKPhA,XqIjpzF_Iq4c05mQ3t8f5R7RUyrWxJVdqL3AoyiyUxg9uI...,FingerLicker,4293,1656310027000,282
72,BgdX6Z51CIbwn1sYyjGKA7oMZcxouUDzCeO_OXRQ_tkV-qk,wyhlWY6DadF0ycxWhpHU672eQtrWYCkw2DDRmNGGnlVGIg,oJObnD4hwTctvj443wvCmZ89lElu-mR-5jy8YT1KK7_zwO...,TORONTOTOKYO,626,1656281795000,458
73,Eq_CljTi_Y8j1dFJ-rbumw0fQWF9Z45BTiZk4y6XTonW5MY,rqdNcntbVbnWo7aDM7Tml1OC4-HEmz-OhDNckky81CZ7BA,aYMQ0HjoMOURgWcc1NKogqLfUvAUQGKQb58HGfBvPZhuC1...,SpicyAppies,3591,1656304076000,37
74,ftxj-IEnBmHGMt0LYcyELwLo5nJOywqfe74I8L1p7rAEm4U,wPKWRj2docYfNphEz0ygYKqtdbX5frmPGy1AGAJJUTrszA,79aTzBZJP4dHgMQm4-bvQu73YjZoMTIj1ROjimuGnbE5Ck...,bertasaurus,3860,1656279487000,289


In [131]:
summoners_df = summoners_challengers_df.merge(challengers_df, left_on='id', right_on='summonerId')

In [132]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,BcLHPFVnuMFT_g6cJdB7LsDocnWKJ3TQyp9mMIY7amMcq-...,St_wL0UOxf_11zObuZn-7FE-yGRGjfjFaeHlAbCMCESgyq...,Spethom,4904,1656316118401,197,e34rJaogiO4CE8zLS8ewLMFGPGoNIMIMsB7gDqVH9aPNHIuY,Spethom,860,I,112,55,False,False,False,False
1,rCmZ-jOb9GFbAkeHOF72Dx8H5lgTi-hyvCx2OIx9vNnMCKg,72e-TFAU4NDDJv0ewfeiKQGwD6Xp1Ty9RvNhzzvrqxz6bA,X_u88wM67NMBXYLCbCAGNCx_19T7pat-j71dhydlbPYF19...,dallasyes,7,1656311568160,53,rCmZ-jOb9GFbAkeHOF72Dx8H5lgTi-hyvCx2OIx9vNnMCKg,dallasyes,677,I,149,125,False,False,True,True
2,5PP1PByW13OTIHNs8f_MQbI2utzf-CKbFa3imrCUD-5JM-k,CE4kJ6tt7PwB4X24Y8CXS4pV73kqdSvJ0wWYyFi_ypioqA,SYeHG18d6HblklTVFAcDoGsAXWwTq1ZeflhRbaztiP900s...,prestivent,4026,1656296968000,128,5PP1PByW13OTIHNs8f_MQbI2utzf-CKbFa3imrCUD-5JM-k,prestivent,857,I,202,142,True,False,False,True
3,F076SSJmWsKsPULh8XD2yr_CcrGzmberIL9oH6WiQ4sydzT6,5CXmlT14PxMEPUOluZZ6VnG0iyHHkcLFtbhFSdpDzWUNWQ...,miECR0OOaJlxGXdFIC3_icU8aPZ9SCTmzrQuuMkLvo64Zx...,Dıshsoap,4368,1656302298000,17,F076SSJmWsKsPULh8XD2yr_CcrGzmberIL9oH6WiQ4sydzT6,Dıshsoap,1483,I,162,58,True,False,False,True
4,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,vNJAtsWbXGLyDPv5HWK6T2VsVvgaY3O7LOUNvL23vJt1-Q,hkhRHja9YNEGxs-c2YZxHJmvTcZaPQSaAMHWnbYt0qwlEC...,Pawnup,659,1656311139000,140,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,Pawnup,709,I,161,108,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,hpVP56ElzFpI9hJtd7l53WcBMs8s2kvqDEiWJnGCJVyLnSg,4Yvk3S970wp_QRtVv1yJ-TgydOOk_qZHnO_LscPrurKPhA,XqIjpzF_Iq4c05mQ3t8f5R7RUyrWxJVdqL3AoyiyUxg9uI...,FingerLicker,4293,1656310027000,282,hpVP56ElzFpI9hJtd7l53WcBMs8s2kvqDEiWJnGCJVyLnSg,FingerLicker,677,I,169,114,False,False,True,True
72,BgdX6Z51CIbwn1sYyjGKA7oMZcxouUDzCeO_OXRQ_tkV-qk,wyhlWY6DadF0ycxWhpHU672eQtrWYCkw2DDRmNGGnlVGIg,oJObnD4hwTctvj443wvCmZ89lElu-mR-5jy8YT1KK7_zwO...,TORONTOTOKYO,626,1656281795000,458,BgdX6Z51CIbwn1sYyjGKA7oMZcxouUDzCeO_OXRQ_tkV-qk,TORONTOTOKYO,560,I,164,134,False,False,True,False
73,Eq_CljTi_Y8j1dFJ-rbumw0fQWF9Z45BTiZk4y6XTonW5MY,rqdNcntbVbnWo7aDM7Tml1OC4-HEmz-OhDNckky81CZ7BA,aYMQ0HjoMOURgWcc1NKogqLfUvAUQGKQb58HGfBvPZhuC1...,SpicyAppies,3591,1656304076000,37,Eq_CljTi_Y8j1dFJ-rbumw0fQWF9Z45BTiZk4y6XTonW5MY,SpicyAppies,927,I,136,58,False,False,False,False
74,ftxj-IEnBmHGMt0LYcyELwLo5nJOywqfe74I8L1p7rAEm4U,wPKWRj2docYfNphEz0ygYKqtdbX5frmPGy1AGAJJUTrszA,79aTzBZJP4dHgMQm4-bvQu73YjZoMTIj1ROjimuGnbE5Ck...,bertasaurus,3860,1656279487000,289,ftxj-IEnBmHGMt0LYcyELwLo5nJOywqfe74I8L1p7rAEm4U,bertasaurus,907,I,152,101,True,False,False,False


In [140]:
summoners_df.iloc[0]['puuid']

'St_wL0UOxf_11zObuZn-7FE-yGRGjfjFaeHlAbCMCESgyqgS636iSJhu5qxxDqBq7XuNlL4I161r8g'

In [177]:
import random

current_matches = watcher.match.by_puuid(my_region, summoners_df.iloc[random.randrange(len(summoners_df))]['puuid'], count=999)

In [178]:
for index, summoner in summoners_df.iterrows():
    current_matches = watcher.match.by_puuid(
        my_region, summoner['puuid'], count=50)

    matches_asset = json.loads('[]')
    for idx, match in enumerate(current_matches[:]):
        match_detail = watcher.match.by_id(my_region, match)
        matches_asset.append(match_detail)

        time.sleep(1)

    write_json(matches_asset, filename='matches_detail'+'_'+summoner['name'])


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [218]:
def load_matches(df):
    matches_asset = []
    for _, summoner in df.iterrows():
        match_asset = read_json(
            filename='matches_detail'+'_'+summoner['name'])
        if match_asset != None:
            matches_asset.extend(match_asset)

    return matches_asset


In [ ]:
matches_asset = load_matches(summoners_df)

In [205]:
from datetime import date

date(2022, 6, 9)

datetime.date(2022, 6, 9)

In [219]:
len(matches_asset)

1740

In [228]:
matches_asset[0]['metadata']['match_id'] #['participants'][0]['units']

'NA1_4355923156'

In [229]:
matches_id = [match['metadata']['match_id'] for match in matches_asset]

In [231]:
len(matches_id)

1740

In [233]:
import collections
duplicated_matches = [item for item, count in collections.Counter(matches_id).items() if count > 1]

In [235]:
len(duplicated_matches)

426

In [236]:
no_dupes = [x for n, x in enumerate(matches_id) if x not in matches_id[:n]]

In [238]:
len(no_dupes)

1026

In [239]:
seen = set()
uniq = [x for x in matches_id if x not in seen and not seen.add(x)]    

In [240]:
len(uniq)

1026

In [247]:
set(['NA1_4355336219','NA1_43550949118']) - set(uniq)

{'NA1_43550949118'}

In [ ]:
tft_assets = read_json(filename='tft_assets')
my_matches = read_json(filename='my_matches'+'_'+my_summoner)
last_match = read_json(filename='last_match')
challengers = read_json(filename='summoners_challengers')
matches_assert = read_json(filename='matches_detail'+'_'+my_summoner)